# Web Scraping — Part 1

*Inspired by web scraping lessons from [Lauren Klein](https://github.com/laurenfklein/emory-qtm340/blob/master/notebooks/class4-web-scraping-complete.ipynb) and [Allison Parrish](https://github.com/aparrish/dmep-python-intro/blob/master/scraping-html.ipynb)*

In this series of lessons, we're going to introduce how to "scrape" data from the internet with the Python libraries requests and BeautifulSoup.

We will cover how to:

* Programmatically access the text of a web page
* Understand the basics of HTML
* Extract certain HTML elements

## Why Do We Need To Scrape At All?

To understand the significance of web scraping, let's walk through the likely data collection process behind [“Film Dialogue from 2,000 screenplays, Broken Down by Gender and Age”](https://pudding.cool/2017/03/film-dialogue/).

To find their 2,000 screenplays, Hannah Andersen and Matt Daniels consulted a number of already existing sources — one of which was the [Cornell Movie Dialogues Corpus](http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html). This is a corpus created by Cornell CIS professors Cristian Danescu-Niculescu-Mizil and Lillian Lee for their paper ["Chameleons in imagined conversations"](http://www.cs.cornell.edu/~cristian/papers/chameleons.pdf). Go Big Red!

These researchers helpfully shared a dataset of every URL that they used to find and access the screenplays in their own project.

**Import Pandas**

In [15]:
import pandas as pd

**Read in CSV file**

In [16]:
urls = pd.read_csv("../data/cornell-movie-corpus/raw_script_urls.csv", delimiter='\t', encoding='utf=8')

**Display DataFrame**

In [17]:
urls

,id,movie_title,script_url
0,m0,10 things i hate about you,http://www.dailyscript.com/scripts/10Things.html
1,m1,1492: conquest of paradise,http://www.hundland.org/scripts/1492-Conquest...
2,m2,15 minutes,http://www.dailyscript.com/scripts/15minutes....
3,m3,2001: a space odyssey,http://www.scifiscripts.com/scripts/2001.txt
4,m4,48 hrs.,http://www.awesomefilm.com/script/48hours.txt
...,...,...,...
612,m612,watchmen,http://www.scifiscripts.com/scripts/wtchmn.txt
613,m613,xxx,http://www.dailyscript.com/scripts/xXx.txt
614,m614,x-men,http://www.scifiscripts.com/scripts/xmenthing...
615,m615,young frankenstein,http://www.horrorlair.com/scripts/young.txt


Each movie title in this CSV file is paired with a URL for the screenplay. How can we actually use these URLs to get computationally tractable text data?

Though we could manually navigate to each URL and copy/paste each screenplay into a file, that would be suuuuper slow and painstaking, and we would lose crucial data in the process — information that might help us automatically distinguish the title of the movie from the screenplay itself, for example. It would be much better to programmatically access the text data attached to every URL.

## Responses and Requests

To programmatically access the text data attached to every URL, we can use a Python library called [requests](https://requests.readthedocs.io/en/master/).

When you type in a URL in your search address bar, you're sending an HTTP **request** for a web page, and the server which stores that web page will accordingly send back a **response**, some web page data that your browser will render.

<img src="../images/request-response.png" class="center">

### Import Requests 

In [9]:
import requests

### Get HTML Data

With the `.get()` method, we can request to "get" web page data for a specific URL, which we will store in a varaible called `response`.

In [106]:
response = requests.get("http://www.scifiscripts.com/scripts/Ghostbusters.txt")

### HTTP Status Code

If we check out `response`, it will simply tell us its [HTTP response code](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status), aka whether the request was successful or not.

"200" is a successful response, while "404" is a common "Page Not Found" error.

In [107]:
response

<Response [200]>

Let's see what happens if we change the title of the movie from *Ghostbusters* to *Ghostboogers* in the URL...

In [108]:
bad_response = requests.get("http://www.scifiscripts.com/scripts/Ghostboogers.txt")

In [109]:
bad_response

<Response [404]>

<img src="../images/Ghostboogers.png" class="center" >

### Extract Text From Web Page

To actually get at the text data in the reponse, we need to use `.text`, which we will save in a variable called `html_string`. The text data that we're getting is formatted in the HTML markup language, which we will talk more about in the BeautifulSoup section below.

In [113]:
html_string = response.text

Here's the screenplay now in a variable.

In [76]:
print(html_string)

						Ghostbusters

							by
						Harold Ramis
							and
						Dan Aykroyd

					Final Shooting Script
				Last revised October 7, 1983


											FADE IN

EXT. NEW YORK PUBLIC LIBRARY -- DAY

The sun shines brightly on the classic facade of the main library at Fifth
Avenue and 42nd Street.  In the adjacent park area, pretty hustlers and
drug peddlers go about their business.

FRONT STEPS

A few people lounge on the steps flanked by the familiar stone lions.

INT. MAIN READING ROOM -- DAY

People are dotted throughout the room sitting at the long oak tables
polished by decades of use.  Reading lamps with green glass shades cast a
golden glow on the tables.  The patina of age is everywhere.  It is very
quiet.

LIBRARIAN

A slightly stout, studious looking girl in her late twenties circulates
quietly among the tables picking up books and putting them on her cart.
Everything seems completely normal and peaceful.

POV

A single eerie musical note signals the presence of something stra

### Extract Text From Multiple Web Pages

Let's quickly demonstrate how we might extract the screenplay text for every URL in the DataFrame. To do so, we're going to create a smaller DataFrame from the Cornell Movie Dialogue Corpus, which consists of the first 10 movies.

In [4]:
sample_urls = urls[:10]

In [5]:
sample_urls

,id,movie_title,script_url
0,m0,10 things i hate about you,http://www.dailyscript.com/scripts/10Things.html
1,m1,1492: conquest of paradise,http://www.hundland.org/scripts/1492-Conquest...
2,m2,15 minutes,http://www.dailyscript.com/scripts/15minutes....
3,m3,2001: a space odyssey,http://www.scifiscripts.com/scripts/2001.txt
4,m4,48 hrs.,http://www.awesomefilm.com/script/48hours.txt
5,m5,the fifth element,http://www.scifiscripts.com/scripts/5thelemen...
6,m6,8mm,http://www.dailyscript.com/scripts/eight-mill...
7,m7,a nightmare on elm street 4: the dream master,http://www.hundland.org/scripts/A-Nightmare-o...
8,m8,a nightmare on elm street: the dream child,http://www.hundland.org/scripts/A-Nightmare-o...
9,m9,the atomic submarine,http://leonscripts.tripod.com/scripts/ATOMICS...


We're going to make a function called `scrape_screenplay()` that includes our `requests.get()` and `response.text` code.

In [6]:
def scrape_screenplay(url):
    response = requests.get(url)
    html_string = response.text
    return html_string

Then we're going apply it to the "script_url" column of the DataFrame and create a new column for the resulting extracted text.

In [ ]:
sample_urls['text'] = sample_urls['script_url'].apply(scrape_screenplay)

In [12]:
sample_urls

,id,movie_title,script_url,text
0,m0,10 things i hate about you,http://www.dailyscript.com/scripts/10Things.html,<html>\r\n<head>\r\n<title>10 Things I Hate Ab...
1,m1,1492: conquest of paradise,http://www.hundland.org/scripts/1492-Conquest...,<h1>404: Not Found</h1>\n
2,m2,15 minutes,http://www.dailyscript.com/scripts/15minutes....,<html>\r\n<head>\r\n<title>15 Minutes script b...
3,m3,2001: a space odyssey,http://www.scifiscripts.com/scripts/2001.txt,Note from poster to Kubrick newsgroup:\r\n\r\n...
4,m4,48 hrs.,http://www.awesomefilm.com/script/48hours.txt,"\n\t\t\t\t\t""48 HRS.""\n\n\t\t\t\t\t by\n\n\t..."
5,m5,the fifth element,http://www.scifiscripts.com/scripts/5thelemen...,\t\t\t\tThe Fifth Element\r\n\r\n\t\t\t\tAn or...
6,m6,8mm,http://www.dailyscript.com/scripts/eight-mill...,"<html>\n\n<head>\n <title>""8MM"", by Andrew K..."
7,m7,a nightmare on elm street 4: the dream master,http://www.hundland.org/scripts/A-Nightmare-o...,<h1>404: Not Found</h1>\n
8,m8,a nightmare on elm street: the dream child,http://www.hundland.org/scripts/A-Nightmare-o...,<h1>404: Not Found</h1>\n
9,m9,the atomic submarine,http://leonscripts.tripod.com/scripts/ATOMICS...,<html>\r\n\r\n<head>\r\n <title>ATOMIC SUBMA...


The DataFrame above is truncated, so we can't see the full contents of the "text" column. But if we print out every row in the column, we can see that we successfully extracted text for each URL (though some of these URLs returned 404 errors).

In [25]:
for text in sample_urls['text']:
    print(text)

<html>
<head>
<title>10 Things I Hate About You script by Karen McCullah Lutz, Kirsten Smith & William Shakespeare</title>
<meta name="description" content="10 Things I Hate About You script by Karen McCullah Lutz, Kirsten Smith & William Shakespeare">
<meta name="keywords" content="10 Things I Hate About You script, 10 Things I Hate About You movie script, 10 Things I Hate About You film script, 10 Things I Hate About You screenplay">
<meta http-equiv="content-type" content="text/html; charset=iso-8859-1">
<meta name="robots" content="index,follow">
<meta http-equiv="Content-Language" content="EN">
<meta name="objecttype" content="Document">
<meta name="Subject" content="Movie scripts, Film scripts">
<meta name="rating" content="General">
<meta name="distribution" content="GLOBAL">
<meta name="revisit-after" content="14 days">
<META NAME="AUTHOR" CONTENT="Daily Script">
<META NAME="OPERATOR" CONTENT="The Daily Script">
<META NAME="intsearch" content="dailyscript.com, daily script, the

## Web Scraping

Not all web pages will be as easy to scrape as these screenplay files, however. Let's say we wanted to scrape the lyrics for Missy Elliott's song "The Rain (Supa Dupa Fly)" (1997) from Genius.com.

<img src="../images/Missy-Elliott.png" class="center" >

Even at a glance, we can tell that this *Genius* web page is a lot more complicated than the *Ghostbusters* page and that it contains a lot of information beyond the lyrics.

Sure enough, if we use our requests library again and try to grab the data for this web page, the underlying data is much more complicated, too.

In [116]:
response = requests.get("https://genius.com/Missy-elliott-the-rain-supa-dupa-fly-lyrics")
html_string = response.text
print(html_string)



<!DOCTYPE html>
<html class="snarly apple_music_player--enabled bagon_song_page--enabled song_stories_public_launch--enabled react_forums--disabled" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" lang="en" xml:lang="en">
  <head>
    <base target='_top' href="//genius.com/">

    <script type="text/javascript">
//<![CDATA[

  var _sf_startpt=(new Date()).getTime();
  if (window.performance && performance.mark) {
    window.performance.mark('parse_start');
  }

//]]>
</script>

<title>Missy Elliott – The Rain (Supa Dupa Fly) Lyrics | Genius Lyrics</title>

<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta content='width=device-width,initial-scale=1' name='viewport'>

  <meta name="apple-itunes-app" content="app-id=709482991">

<link href="https://assets.genius.com/images/apple-touch-icon.png?1582666308" rel="apple-touch-icon" />


  

  <link href="https://assets.genius.com/images/apple-touch-icon.png?1582666308" rel="apple-

How can we extract just the song lyrics from this messy soup of a document? Luckily there's a Python library that can help us called BeautifulSoup, which parses HTML documents.

To understand BeautifulSoup and HTML, we're going to briefly depart from our Missy Elliot lyrics challenge to consider a much simpler website. This toy website was made by the poet, programmer, and professor Allison Parrish explicitly for the purposes of teaching BeautifulSoup.

## HTML

Parrish's website is titled "Kittens and the TV Shows They Love." It can be found at the following URL: http://static.decontextualize.com/kittens.html

<img src="../images/kittens-web.png" class="center" >

If we use our requests library on this Kittens TV website, this is what we get:

In [32]:
response = requests.get("http://static.decontextualize.com/kittens.html")
html_string = response.text
print(html_string)

<!doctype html>
<html>
	<head>
		<title>Kittens!</title>
		<style type="text/css">
			span.lastcheckup { font-family: "Courier", fixed; font-size: 11px; }
		</style>
	</head>
	<body>
		<h1>Kittens and the TV Shows They Love</h1>
		<div class="kitten">
			<h2>Fluffy</h2>
			<div><img src="http://placekitten.com/120/120"></div>
			<ul class="tvshows">
				<li>
					<a href="http://www.imdb.com/title/tt0106145/">Deep Space Nine</a>
				</li>
				<li>
					<a href="http://www.imdb.com/title/tt0088576/">Mr. Belvedere</a>
				</li>
			</ul>
			Last check-up: <span class="lastcheckup">2014-01-17</span>
		</div>
		<div class="kitten">
			<h2>Monsieur Whiskeurs</h2>
			<div><img src="http://placekitten.com/110/110"></div>
			<ul class="tvshows">
				<li>
					<a href="http://www.imdb.com/title/tt0106179/">The X-Files</a>
				</li>
				<li>
					<a href="http://www.imdb.com/title/tt0098800/">Fresh Prince</a>
				</li>
			</ul>
			Last check-up: <span class="lastcheckup">2013-11-02</span>
		</div

### HTML Tags

This is an HTML document. HTML stands for HyperText Markup Language. It is the standard language for writing web page documents. The most important thing you need to know about HTML is that the language uses HTML "tags" to represent different elements, such as a main header `<h1>`. 

| HTML Tag                | Explanation                              |
|--------------------|-------------------------------------------|
| <\!DOCTYPE>        | Defines document type                 |
| <html\>             | Defines HTML document                  |
| <head\>             | Main information about document    |
| <title\>            | Title for document          |
| <body\>             | Document body               |
| <h1\> to <h6\>       |  Headings                    |
| <p\>                | Paragraph                       |
| <br\>               | Line break               |
| <\!\-\-comment here-\-> | Comment                         |
| <img\> | Image                         |
| <a\> | Hyperlink                       |
| <ul\> | Unordered list                     |
| <ol\> | Ordered list                     |
| <li\> | List item                     |
| <style\> | Style information for a document                    |
| <div\> | Section in a document                   |
| <span\> | Section in a document                   |

HTML tags often, but not always, require a "closing" tag. For example, the main header "Kittens and the TV Shows They Love" will be surrounded by `<h1>` (opening tag) and `</h1>` (closing tag) on either side: `<h1>Kittens and the TV Shows They Love</h1>`

### HTML Attributes, Classes, and IDs

HTML elements sometimes come with even more information inside a tag. This will often be a keyword (like `class` or `id`) followed by an equals sign `=` and a further descriptor such as `<div class="kitten">`

We need to know about tags as well as attributes, classes, and IDs because this is how we're going to extract specific HTML data with BeautifulSoup.

## BeautifulSoup

In [18]:
from bs4 import BeautifulSoup

To make a BeautifulSoup document, we call `BeautifulSoup()` with two parameters: the `html_string` from our HTTP request and [the kind of parser](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#specifying-the-parser-to-use) that we want to use, which will always be `"html.parser"` for our purposes.

In [13]:
response = requests.get("http://static.decontextualize.com/kittens.html")
html_string = response.text

document = BeautifulSoup(html_string, "html.parser")

In [145]:
document

<!DOCTYPE doctype html>

<html>
<head>
<title>Kittens!</title>
<style type="text/css">
			span.lastcheckup { font-family: "Courier", fixed; font-size: 11px; }
		</style>
</head>
<body>
<h1>Kittens and the TV Shows They Love</h1>
<div class="kitten">
<h2>Fluffy</h2>
<div><img src="http://placekitten.com/120/120"/></div>
<ul class="tvshows">
<li>
<a href="http://www.imdb.com/title/tt0106145/">Deep Space Nine</a>
</li>
<li>
<a href="http://www.imdb.com/title/tt0088576/">Mr. Belvedere</a>
</li>
</ul>
			Last check-up: <span class="lastcheckup">2014-01-17</span>
</div>
<div class="kitten">
<h2>Monsieur Whiskeurs</h2>
<div><img src="http://placekitten.com/110/110"/></div>
<ul class="tvshows">
<li>
<a href="http://www.imdb.com/title/tt0106179/">The X-Files</a>
</li>
<li>
<a href="http://www.imdb.com/title/tt0098800/">Fresh Prince</a>
</li>
</ul>
			Last check-up: <span class="lastcheckup">2013-11-02</span>
</div>
</body>
</html>

### Extract HTML Element

We can use the `.find()` method to find and extract certain elements, such as a main header.

In [118]:
document.find("h1")

<h1>Kittens and the TV Shows They Love</h1>

If we want only the text contained between those tags, we can use `.text` to extract just the text.

In [146]:
document.find("h1").text

'Kittens and the TV Shows They Love'

In [147]:
type(document.find("h1").text)

str

Find the HTML element that contains an image.

In [120]:
document.find("img")

<img src="http://placekitten.com/120/120"/>

### Extract Multiple HTML Elements

You can also extract multiple HTML elements at a time with `.find_all()`

In [121]:
document.find_all("img")

[<img src="http://placekitten.com/120/120"/>,
 <img src="http://placekitten.com/110/110"/>]

In [122]:
document.find_all("div", attrs={"class": "kitten"})

[<div class="kitten">
 <h2>Fluffy</h2>
 <div><img src="http://placekitten.com/120/120"/></div>
 <ul class="tvshows">
 <li>
 <a href="http://www.imdb.com/title/tt0106145/">Deep Space Nine</a>
 </li>
 <li>
 <a href="http://www.imdb.com/title/tt0088576/">Mr. Belvedere</a>
 </li>
 </ul>
 			Last check-up: <span class="lastcheckup">2014-01-17</span>
 </div>, <div class="kitten">
 <h2>Monsieur Whiskeurs</h2>
 <div><img src="http://placekitten.com/110/110"/></div>
 <ul class="tvshows">
 <li>
 <a href="http://www.imdb.com/title/tt0106179/">The X-Files</a>
 </li>
 <li>
 <a href="http://www.imdb.com/title/tt0098800/">Fresh Prince</a>
 </li>
 </ul>
 			Last check-up: <span class="lastcheckup">2013-11-02</span>
 </div>]

In [125]:
document.find("h2").text

'Fluffy'

In [148]:
document.find_all("h2")

[<h2>Fluffy</h2>, <h2>Monsieur Whiskeurs</h2>]

``` {warning}
Heads up! The code below will cause an error.
```

Let's try to extract the text from all the header2 elements:

In [70]:
document.find_all("h2").text

AttributeError: ResultSet object has no attribute 'text'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

Uh oh. That didn't work! In order to extract text data from multiple HTML elements, we need a `for` loop and some list-building.

In [15]:
all_h2_headers = document.find_all("h2")

In [128]:
all_h2_headers

[<h2>Fluffy</h2>, <h2>Monsieur Whiskeurs</h2>]

First we will make an empty list called `h2_headers`.

Then `for` each `header` in `all_h2_headers`, we will grab the `.text`, put it into a variable called `header_contents`, then `.append()` it to our `h2_headers` list.

In [16]:
h2_headers = []
for header in all_h2_headers:
    header_contents = header.text
    h2_headers.append(header_contents)

In [17]:
h2_headers

['Fluffy', 'Monsieur Whiskeurs']

````{admonition} Python Review: List Comprehensions
:class: pythonreview
How might we transform this for loop into a *list comprehension*?
```
#For Loop
h2_headers = []
for header in all_h2_headers:
    header_contents = header.text
    h2_headers.append(header_contents)
```
````

**Check out the *list comprehension* answer here**

In [18]:
h2_headers = [header.text for header in all_h2_headers]
h2_headers

['Fluffy', 'Monsieur Whiskeurs']

## Inspect HTML Elements with Browser

Most times if you're looking to extract something from an HTML document, it's best to use your "Inspect" capabilities in your web browser. You can hover over elements that you're interested in and find that specific element in the HTML.

<img src="../images/inspect.png" class="center">

For example, if we hover over the main header:

<img src="../images/inspect-h1.png" class="center" >

## Your Turn!

Ok so now we've learned a little bit about how to use BeautifulSoup to parse HTML documents. So how would we apply what we've learned to extract Missy Elliott lyrics?

In [20]:
response = requests.get("https://genius.com/Missy-elliott-the-rain-supa-dupa-fly-lyrics")
html_str = response.text

document = BeautifulSoup(html_str, "html.parser")

In [21]:
document


<!DOCTYPE html>

<html class="snarly apple_music_player--enabled bagon_song_page--enabled song_stories_public_launch--enabled react_forums--disabled" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<base href="//genius.com/" target="_top"/>
<script type="text/javascript">
//<![CDATA[

  var _sf_startpt=(new Date()).getTime();
  if (window.performance && performance.mark) {
    window.performance.mark('parse_start');
  }

//]]>
</script>
<title>Missy Elliott – The Rain (Supa Dupa Fly) Lyrics | Genius Lyrics</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="app-id=709482991" name="apple-itunes-app"/>
<link href="https://assets.genius.com/images/apple-touch-icon.png?1597864036" rel="apple-touch-icon"/>
<link href="https://assets.genius.com/images/apple-touch-icon.png?1597864036" rel="apple-touch-icon"/>
<!-- Mobil

https://genius.com/Missy-elliott-the-rain-supa-dupa-fly-lyrics

What HTML element do we need to "find" to extract the song lyrics?

**Check answer here**

In [22]:
missy_lyrics = document.find("p").text
print(missy_lyrics)

[Intro: Missy Elliott & Timbaland]
[*Yawning*]
Run the track

[Chorus: Missy Elliott & Ann Peebles]
Me I'm super fly, super dupa fly
(I can't stand the rain) Supa dupa fly
Me I'm super fly, (against my window) super dupa fly
(I can't stand the rain) supa dupa fly
Me I'm super fly, (against my window) super dupa fly
(I can't stand the rain) supa dupa fly
Me I'm super fly, (against my window)

[Verse 1: Missy Elliott]
When the rain hits my window
I take and *cough* me some indo
Me and Timbaland, ooh, we sang a jangle
We so tight that you get our styles tangled
Sway on dosie-do like you loco
Can we get kinky tonight?
Like Coko, so-so
You don't wanna play with my Yo-Yo
I smoke my hydro on the D-low (D-D-D-D-D-low)

[Hook: Ann Peebles]
I can't stand the rain against my window
I can't stand the rain against my window
I can't stand the rain against my window
I can't stand the rain against my window
I can't stand the rain

[Verse 2: Missy Elliott]
Beep, beep, who got the keys to the Jeep, vroo

What HTML element do we need to "find" to extract the title?

**Check answer here**

In [23]:
song_title = document.find('h1').text
print(song_title)

The Rain (Supa Dupa Fly)
